## Scraping video data using YouTube API

#### Set-up

Run the following instructions on your terminal:
``` bash
pip install google-api-python-client
pip install --upgrade google-api-python-client
```

In [1]:
from googleapiclient.discovery import build
from IPython.display import JSON
from pprint import pprint
import pandas as pd
import json

import os
os.chdir(os.path.expanduser("../"))

from dees_package.youtube_functions import *

print("Current working directory:", os.getcwd())

Current working directory: c:\Users\soong\Documents\LSE_Acads\AT_23.24\DS105A\final_project\ds105a-project-dees-nuts


In [2]:
credentials_file_path = "./credentials.json"

# open the file and load the data into a variable
with open(credentials_file_path, "r") as f:
    credentials = json.load(f)

In [3]:
# creating service object of the youtube version 3 API
my_created_youtube = build('youtube', 'v3', developerKey=credentials['youtube_api_key'])

## Getting a list of music videos so that we have a list of videos to analyse


Carry out test search using `youtube_search`

In [4]:
youtube_search_data, mv_video_id = youtube_search(my_created_youtube, 1000, "official music video", "US", "video", 10)

with open('./data/yt_search_data.json', 'w') as json_file:
    json.dump(youtube_search_data, json_file, indent=4)

yt_search_df = pd.DataFrame(youtube_search_data)

In [5]:
yt_search_df.to_csv('./data/test_num.csv')

## From the list of music videos, get statistics on each video


In [6]:
mv_video_stats = get_stats(my_created_youtube, mv_video_id)
with open('./data/mv_stats_data.json', 'w') as json_file:
    json.dump(mv_video_stats, json_file, indent=4)

mv_videostats_df = pd.DataFrame(mv_video_stats)

HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/videos returned "The request specifies an invalid filter parameter.". Details: "[{'message': 'The request specifies an invalid filter parameter.', 'domain': 'youtube.parameter', 'reason': 'invalidFilters', 'location': 'parameters.', 'locationType': 'other'}]">

In [ ]:
# merge the mv stats and search dataframes
searchandstats_merged_df = pd.merge(yt_search_df, mv_videostats_df, left_on='video_id', right_on='video_id')
searchandstats_merged_df.to_json('./data/merged.json')

searchandstats_merged_df.to_csv('./data/merged.csv')

In [ ]:
comments_df = get_comments_in_videos(my_created_youtube, mv_video_id)

In [ ]:
comments_df.head()

In [ ]:
deets_df = get_video_details(my_created_youtube, mv_video_id)

In [ ]:
deets_df.head()